In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn


class LitModule(pl.LightningModule):
  
  def __init__(self, classCount, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.module = ae_cnn_models.AECNN(classCount)
    self.classCount = classCount
    # ... more initialization elided for simplicity

    self.loss_center = CenterLoss()
    self.loss1 = torch.nn.CrossEntropyLoss()
    self.loss2 = torch.nn.CrossEntropyLoss()
    self.loss3 = torch.nn.CrossEntropyLoss()
    self.loss4 = torch.nn.CrossEntropyLoss()
    self.loss5 = torch.nn.CrossEntropyLoss()
    self.loss6 = torch.nn.CrossEntropyLoss()
    self.loss7 = torch.nn.CrossEntropyLoss()

  def configure_optimizers(self):
    # ... elided for simplicity

  def train_dataloader(self):
    # ... elided for simplicity

  def training_step(self, batch, batch_idx):
    x, y = batch
    input = self.trans_train(x)
    input = input.to(dtype=torch.float32, device=self.device)
    varInput = torch.autograd.Variable(input)
    varTarget1 = torch.autograd.Variable(input)
    varTarget2 = torch.autograd.Variable(y)
    varOutput1, varOutput2 = self.module.forward(varInput)
    classifierOut1, classifierOut2 = varOutput2

    # autoencoder model loss:
    lossvalue1 = self.loss1(varOutput1, varTarget1)
    # weighting between main and aux branch of inception model:
    lossvalue2 = 0.8 * self.loss2(classifierOut1, varTarget2) + 0.2 * self.loss2(classifierOut2, varTarget2)  

    losses = [loss_center,loss_val1,loss_val2,loss_val3,loss_val4,loss_val5,loss_val6,loss_val7]
    # weighting btween MSE and BCE respectively:
    loss = sum(losses)

    output = {
      'loss': loss,  # required
    }
    return output

def trans_train(self, x):
  # ...  some image transformations elided for simplicity



class CenterLoss(nn.Module):
    """Center loss.
    
    Reference:
    Wen et al. A Discriminative Feature Learning Approach for Deep Face Recognition. ECCV 2016.
    
    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """
    def __init__(self, num_classes=10, feat_dim=2, use_gpu=True):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.use_gpu = use_gpu

        if self.use_gpu:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())
        else:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())

        classes = torch.arange(self.num_classes).long()
        if self.use_gpu: classes = classes.cuda()
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = distmat * mask.float()
        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size

        return loss


if __name__ == "__main__":
    main():
  classCount = 14
  model = LightningChestXrayCnnClassifierInceptionV3(classCount)
  trainer = pl.Trainer(max_epochs=15)
  trainer.fit(model)


